In [1]:
## il problema del commesso viaggiatore applicato alle città della Sicilia
## anche se i dati sono poche decine di byte, l'elaborazione richiede un tempo relativamente lungo
## se si vogliono provare tutti gli itinerari (forza bruta)

## lista delle etichette
a = ['AG','CL','CT','EN','ME','PA','RG','SR','TP'] ## 9 province
## matrice delle distanze
b = [[0,60,160,80,250,120,160,220,140],             ## distanze da 'AG'
     [0,0,100,20,190,100,140,160,200],              ## distanze da 'CL'
     [0,0,0,80,90,200,100,60,300],                  ## distanze da 'CT'
     [0,0,0,0,170,120,180,140,220],                 ## distanze da 'EN'
     [0,0,0,0,0,220,190,150,320],                   ## distanze da 'ME'
     [0,0,0,0,0,0,300,260,100],                     ## distanze da 'PA'
     [0,0,0,0,0,0,0,100,300],                       ## distanze da 'SR'
     [0,0,0,0,0,0,0,0,360],                         ## distanze da 'RG'
     [0,0,0,0,0,0,0,0,0]                            ## distanze da 'TP'
    ]
for i in range(len(b[0])):
    for j in range(len(b[0])):
        if i > j:
            b[i][j] = b[j][i]
print(b)
tot = 0
for i in range(len(b[0])):
    tot += sum (b[i][:])
print("totale = " + str(tot) )


[[0, 60, 160, 80, 250, 120, 160, 220, 140], [60, 0, 100, 20, 190, 100, 140, 160, 200], [160, 100, 0, 80, 90, 200, 100, 60, 300], [80, 20, 80, 0, 170, 120, 180, 140, 220], [250, 190, 90, 170, 0, 220, 190, 150, 320], [120, 100, 200, 120, 220, 0, 300, 260, 100], [160, 140, 100, 180, 190, 300, 0, 100, 300], [220, 160, 60, 140, 150, 260, 100, 0, 360], [140, 200, 300, 220, 320, 100, 300, 360, 0]]
totale = 12120


In [2]:
%%time
## in questa cella facciamo un esempio ridotto con l'uso della forza bruta
## ma limitandoci a 7 città per motivi di tempo
## ad ogni città aggiunta il tempo si moltiplica di un fattore pari al numero di città
## perché il tempo è proporzionale al fattoriale n! (circa un'ora di elaborazione)
## con questo algoritmo è impensabile provare l'itinerario minimo per tutte le città italiane 

import progressbar
n = len(a)
n = 6    ## si può usare con n = 6 per test rapidi
d = []
y = int(10**n)
bar = progressbar.ProgressBar(maxval=y, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()
for i in range(y):
    bar.update(i)
    x = str(i).zfill(n)
    z = 0
    for j in range(n):
        if (z or (int(x[j]) == 0) or (int(x[j]) > n)):
##            print(x+" "+str(j)+" "+str(x[j]))
            z = 1
            break
        for k in range(j + 1, n):
##            print(str(x)+" "+str(j)+" "+str(k))
            if (x[j] == x[k]):
                z = 1
                break
    if (not z):
##        print("+"+x)
        d.append(int(x))
bar.finish()

CPU times: total: 41.5 s
Wall time: 44.5 s


In [3]:
## (eventuale) caricamento dei dati da file
## (nel caso non si voglia/possa aspettare il risultato dell'elaborazione precedente)
## da usare in alternativa alla cella precedente (o a completamento di un test rapido)
## questo file contiene i dati completi relativi alla Sicilia (corrispondenti a 9! = 362880 itinerari)

import pandas as pd
###filename = "C:\\users\\ACEAN5155875QD\\dropbox\\scuola\\python\\corso python\\lezione 29\\itinerari9.csv"
### filename = "/content/drive/MyDrive/Colab Notebooks/itinerari9.csv"
### modificare il filename con il percorso del file sul vs. computer
filename = "itinerari9.csv"
df = pd.read_csv(filename,header=0)
dz = df.iloc[:].values
d = []
for i in range(len(dz)):
    d.append(dz[i][0])

In [4]:
## esaminiamo i risultati della forza bruta e vediamo qual è l'itinerario più breve in assoluto
print(len(d)) ### stampa il numero di possibili itinerari
print(d[-1])  ### stampa i codici dell’ultimo itinerario (1=”AG”, 2=”CL”, etc.)
z = len(d)
n = len(a)
vmin = 1e6
for i in range(z):
    p = str(d[i]).zfill(n)
    s = 0
    for j in range(1,n):
        s = s + b[int(p[j-1])-1][int(p[j])-1]
    if (s < vmin) or (i == 0):
        vmin = s
        pmin = i
print(str(i) + " " + str(p) + " " + str(s))
q = str(d[pmin])
for i in range(len(q)):
    print(a[int(q[i])-1])
print(str(d[pmin]).zfill(9) + " " + str(vmin))


362880
987654321
362879 987654321 1390
ME
CT
SR
RG
AG
CL
EN
PA
TP
538712469 710


In [5]:
### ora proviamo un algoritmo estremamente più semplice che sceglie di volta in volta
### la città più vicina a quella di partenza od all'ultima città visitata 
### (escludendo di passare due volte per la stessa città)
### qui possiamo visualizzare tutti i risultati perché abbiamo un solo itinerario associato
### ad ogni città di partenza
n = len(a)
c = []
for i in range(n):            ## per ogni città di partenza
    print(a[i])
    c.append([])
    c[i].append(i)
    r = i
    s = 0
    for j in range(n - 1):    ## si devono stabilire le (n-1) tappe successive
        pmin = (r + 1) % n
        vmin = tot
        for k in range(n):    ## scegliendo di volta in volta la tappa più breve
                              ## escludendo le città già visitate e la città di partenza
            if ((k not in c[i]) and (b[r][k] <= vmin)):
                pmin = k
                vmin = b[r][k]
        r = pmin
        c[i].append(r)
        print (str(pmin) + " " + str(a[pmin]) + " " + str(vmin))
        s = s + vmin
    print(a[i] + " " + str(s))
    print("-----------------")


AG
1 CL 60
3 EN 20
2 CT 80
7 SR 60
6 RG 100
4 ME 190
5 PA 220
8 TP 100
AG 830
-----------------
CL
3 EN 20
2 CT 80
7 SR 60
6 RG 100
0 AG 160
5 PA 120
8 TP 100
4 ME 320
CL 960
-----------------
CT
7 SR 60
6 RG 100
1 CL 140
3 EN 20
0 AG 80
5 PA 120
8 TP 100
4 ME 320
CT 940
-----------------
EN
1 CL 20
0 AG 60
5 PA 120
8 TP 100
6 RG 300
7 SR 100
2 CT 60
4 ME 90
EN 850
-----------------
ME
2 CT 90
7 SR 60
6 RG 100
1 CL 140
3 EN 20
0 AG 80
5 PA 120
8 TP 100
ME 710
-----------------
PA
8 TP 100
0 AG 140
1 CL 60
3 EN 20
2 CT 80
7 SR 60
6 RG 100
4 ME 190
PA 750
-----------------
RG
7 SR 100
2 CT 60
3 EN 80
1 CL 20
0 AG 60
5 PA 120
8 TP 100
4 ME 320
RG 860
-----------------
SR
2 CT 60
3 EN 80
1 CL 20
0 AG 60
5 PA 120
8 TP 100
6 RG 300
4 ME 190
SR 930
-----------------
TP
5 PA 100
1 CL 100
3 EN 20
2 CT 80
7 SR 60
6 RG 100
0 AG 160
4 ME 250
TP 870
-----------------


In [ ]:
### come si vede l'itinerario più breve è lo stesso 